In [1]:
import yfinance as yf
import pandas as pd

tickers = [
    "INRG.L", # Global Clean Energy
    "URTH", # MSCI World
    "IEUR", # MSCI Europe
    "EEM", # MSCI Emerging Markets
    "IVE", # S&P 500
]


# Download 
# data = yf.download(tickers, period="3y",auto_adjust=True)['Close']

data = yf.download(tickers, period="3y")

if isinstance(data.columns, pd.MultiIndex):
    adj_close = data['Close']
else:
    adj_close = data[['Close']] # For single ticker cases

C:\Users\lizhiqian\AppData\Local\Temp\ipykernel_32036\1855852346.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="3y")
[*********************100%***********************]  5 of 5 completed


In [2]:
from utils_webscrap import get_ticker_holdings
holdings_info_dict = {}
manual_tickers_list = []
for fund_ticker in tickers:
    info_df = get_ticker_holdings(fund_ticker)
    if info_df is not None:
        holdings_info_dict[fund_ticker] = (info_df)
    else:
        manual_tickers_list.append(fund_ticker)

Error retrieving holdings for INRG.L: No listings found for ticker INRG.L
Holdings Info Retrieved for URTH
Holdings Info Retrieved for IEUR
Holdings Info Retrieved for EEM
Holdings Info Retrieved for IVE


In [3]:
holding_df = holdings_info_dict['URTH']
relevant_eft_columns = ['ticker', 'sector', 'location', 'weight']

for col in relevant_eft_columns:
    if col not in holding_df.columns:
        print(f"Warning: Column '{col}' not found in holdings DataFrame.")

holding_df[relevant_eft_columns]

,ticker,sector,location,weight
0,NVDA,Information Technology,United States,5.45
1,AAPL,Information Technology,United States,4.85
2,MSFT,Information Technology,United States,4.11
3,AMZN,Consumer Discretionary,United States,2.67
4,GOOGL,Communication,United States,2.19
...,...,...,...,...
1341,VGH6,Cash and/or Derivatives,European Union,0.00
1342,ESH6,Cash and/or Derivatives,NaN,0.00
1343,ETD_JPY,Cash and/or Derivatives,Japan,0.00
1344,ETD_GBP,Cash and/or Derivatives,United Kingdom,0.00


In [4]:
from bs4 import BeautifulSoup
from pathlib import Path
from utils_webscrap import extract_subtable


header_dict = {
    "tickers": "top-holdings",
    "location": "countries",
    "sector": "sectors",
}

# Load and parse MSCI.html
html_path = Path("data_webpages/INRG.L.html")
with open(html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, "html.parser")


holding_df = extract_subtable(soup, header_dict["sector"])
holding_df

,Name,Value
0,Versorger,"45,74%"
1,Industrie,"29,37%"
2,Technologie,"12,73%"
3,Grundstoffe,"1,18%"
4,Sonstige,"10,98%"
